# Filter Itemid Input

This script is used for filtering itemids from TABLE INPUTEVENTS.

1. We check number of units of each itemid and choose the major unit as the target of unit conversion.
2. In this step we do not apply any filtering to the data.

## Output

1. itemid of observations for inputevents.
2. unit of measurement for each itemid.

In [2]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool

from utils import getConnection

%matplotlib inline

In [3]:
try:
    conn = getConnection()
    print('Connected to Postgre Database!')
except:
    print('Fail to connect!')
    
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

Connected to Postgre Database!


In [4]:
valid_input = []
valid_input_unit = []

Compare the speed between two ways of query. In order to accelerate, we manually create a TABLE ADMISSION_IDS to store all admission_ids.

In [4]:
cur = conn.cursor()
start = datetime.datetime.now()
for t in range(10):
    cur = conn.cursor()
    cur.execute('select coalesce(amountuom, \'\'), count(*) from mimiciii.inputevents_cv where itemid=30044 and hadm_id in (select * from admission_ids) group by amountuom')
    res = cur.fetchall()
end = datetime.datetime.now()
print(end - start)
print(res)

start = datetime.datetime.now()
for t in range(10):
    cur = conn.cursor()
    cur.execute('select coalesce(amountuom, \'\'), count(*) from mimiciii.inputevents_cv where itemid=30044 and hadm_id in ({0}) group by amountuom'.format(admission_ids_txt))
    res = cur.fetchall()
end = datetime.datetime.now()
print(end - start)
print(res)

0:00:00.160914
[('', 315), ('mg', 160)]
0:00:02.639574
[('mg', 160), ('', 315)]


In [5]:
# inputevents
def stat_inputevents_unit_task(itemid, admission_ids_txt):
    tconn = getConnection()
    tcur = tconn.cursor()
#     tcur.execute('SELECT amountuom, count(amountuom) FROM mimiciii.inputevents_cv \
#                 WHERE amountuom is not null and itemid = '+ str(itemid) +' and hadm_id in ('+admission_ids_txt+') group by amountuom')
#     tcur.execute('select coalesce(amountuom, \'\'), count(*) from (select amountuom, itemid, hadm_id from mimiciii.inputevents_cv union select amountuom, itemid, hadm_id from mimiciii.inputevents_mv) \
#         where itemid={0} and hadm_id in (select hadm_id from admission_ids) group by amountuom'.format(itemid))
    tcur.execute('select amountuom, sum(count::int) from (\
                    select coalesce(amountuom, \'\') as amountuom, count(*) from mimiciii.inputevents_cv where itemid = {0} and hadm_id in (select * from admission_ids) group by amountuom\
                    union all\
                    select coalesce(amountuom, \'\') as amountuom, count(*) from mimiciii.inputevents_mv where itemid = {0} and hadm_id in (select * from admission_ids) group by amountuom\
                    ) as t where amountuom<>\'\' group by amountuom'.format(itemid))
    outputunits = tcur.fetchall()
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ):
        return (itemid, None, None)
    percentage = float(outputunits[0][1]) / total *100.0
    tconn.close()
    return (itemid, percentage, outputunits)

p = Pool()
valid_vupairs = [p.apply_async(stat_inputevents_unit_task, args=(i, admission_ids_txt)) for i in input_itemid]
p.close()
p.join()
valid_vupairs = [x.get() for x in valid_vupairs]

## iterate thru each itemID
For each item id, we count number of observations for each unit of measurement.

For example,
IN 225883 : 98.24 : 3 : [('dose', 16477L), ('mg', 251L), ('grams', 44L)]
This means that for itemid 225883, there are:
1. 16477 records using dose as its unit of measurement.
2. 251 records using mg as its unit of measurement.
3. 44 records using grams as its unit of measurement.

dose has 98.24% over all the observations for this itemid, we can say that dose is a majority unit. 
1. We will keep this itemid because 98% is high. we can relatively safe to discard the observations that has different unit of measurement. i.e. if we discard mg and grams, we lose 251+44 records which is little, compared to 16477 records we can keep.
2. We will record main unit of measurement for this itemID as dose.

In [6]:
valid_vupairs = [x for x in valid_vupairs if x[1] is not None]
valid_vupairs_des = sorted(valid_vupairs, key=lambda x: x[1])
for itemid, percentage, outputunits in valid_vupairs_des:
    print("IN "+str(itemid) + "\t" + "{:.2f}".format(percentage) + "\t" + str(len(outputunits))+" : "+ str(outputunits))
    
np.save('res/filtered_input_raw.npy', {'raw': valid_vupairs})

IN 30174	52.13	2 : [('ml', 171), ('mg', 157)]
IN 30384	66.47	2 : [('ml', 1982), ('mg', 1000)]
IN 225845	69.42	3 : [('dose', 1024), ('mg', 450), ('grams', 1)]
IN 30046	72.65	2 : [('ml', 85), ('mg', 32)]
IN 30069	73.28	2 : [('mg', 1901), ('ml', 693)]
IN 225910	75.12	2 : [('dose', 20669), ('mg', 6847)]
IN 221744	79.95	2 : [('mcg', 69025), ('mg', 17315)]
IN 30089	82.37	2 : [('tsp', 121501), ('gm', 26008)]
IN 30151	84.20	2 : [('mg', 1183), ('ml', 222)]
IN 30135	84.43	2 : [('mg', 1166), ('ml', 215)]
IN 30298	86.73	4 : [('mEQ', 8644), ('U', 800), ('mEq', 502), ('ml', 21)]
IN 225914	87.16	4 : [('mcg', 95), ('dose', 8), ('mg', 4), ('pg', 2)]
IN 225913	88.26	3 : [('dose', 880), ('mg', 116), ('grams', 1)]
IN 30022	89.86	2 : [('mg', 647), ('ml', 73)]
IN 228003	90.41	2 : [('dose', 66), ('mg', 7)]
IN 30148	90.65	2 : [('mg', 291), ('ml', 30)]
IN 30177	92.11	2 : [('mg', 3246), ('ml', 278)]
IN 30114	93.43	2 : [('mg', 25355), ('ml', 1783)]
IN 225866	94.60	2 : [('dose', 526), ('mg', 30)]
IN 30178	94.65	2

IN 42839	100.00	1 : [('ml', 2)]
IN 45131	100.00	1 : [('ml', 9)]
IN 45427	100.00	1 : [('ml', 10)]
IN 43979	100.00	1 : [('ml', 32)]
IN 44305	100.00	1 : [('ml', 1)]
IN 46366	100.00	1 : [('ml', 5)]
IN 42887	100.00	1 : [('ml', 29)]
IN 44962	100.00	1 : [('ml', 7)]
IN 46217	100.00	1 : [('ml', 20)]
IN 46617	100.00	1 : [('ml', 1)]
IN 42204	100.00	1 : [('ml', 252)]
IN 45369	100.00	1 : [('ml', 41)]
IN 44601	100.00	1 : [('ml', 8)]
IN 46783	100.00	1 : [('ml', 7)]
IN 46458	100.00	1 : [('ml', 25)]
IN 44822	100.00	1 : [('ml', 6)]
IN 42888	100.00	1 : [('ml', 126)]
IN 44469	100.00	1 : [('ml', 7)]
IN 42188	100.00	1 : [('ml', 3)]
IN 46106	100.00	1 : [('ml', 14)]
IN 46258	100.00	1 : [('ml', 1)]
IN 45633	100.00	1 : [('ml', 13)]
IN 41556	100.00	1 : [('ml', 5)]
IN 44125	100.00	1 : [('ml', 84)]
IN 41895	100.00	1 : [('ml', 11)]
IN 46115	100.00	1 : [('ml', 23)]
IN 45498	100.00	1 : [('ml', 8)]
IN 42493	100.00	1 : [('ml', 8)]
IN 42758	100.00	1 : [('ml', 3)]
IN 30303	100.00	1 : [('cal', 165273)]
IN 42308	100.00	1 :

IN 40582	100.00	1 : [('ml', 8)]
IN 45744	100.00	1 : [('ml', 129)]
IN 43493	100.00	1 : [('ml', 4)]
IN 44712	100.00	1 : [('ml', 1)]
IN 40581	100.00	1 : [('ml', 19)]
IN 223257	100.00	1 : [('units', 452)]
IN 44303	100.00	1 : [('ml', 4)]
IN 42326	100.00	1 : [('ml', 2)]
IN 30050	100.00	1 : [('mg', 41705)]
IN 46795	100.00	1 : [('ml', 2)]
IN 30142	100.00	1 : [('mg', 14004)]
IN 43002	100.00	1 : [('ml', 1)]
IN 46114	100.00	1 : [('ml', 38)]
IN 42740	100.00	1 : [('ml', 1)]
IN 43506	100.00	1 : [('ml', 77)]
IN 221689	100.00	1 : [('mg', 197)]
IN 44031	100.00	1 : [('ml', 18)]
IN 41846	100.00	1 : [('ml', 193)]
IN 44575	100.00	1 : [('ml', 2)]
IN 42830	100.00	1 : [('ml', 1)]
IN 46351	100.00	1 : [('ml', 12)]
IN 45454	100.00	1 : [('ml', 7)]
IN 42494	100.00	1 : [('ml', 16)]
IN 41373	100.00	1 : [('ml', 2)]
IN 42663	100.00	1 : [('ml', 113)]
IN 45586	100.00	1 : [('ml', 5)]
IN 40596	100.00	1 : [('ml', 24)]
IN 46235	100.00	1 : [('ml', 22)]
IN 42971	100.00	1 : [('ml', 9)]
IN 42115	100.00	1 : [('ml', 9)]
IN 41445	

In [ ]:
conn = getConnection()
sql = 'select hadm_id, amountuom, count(amountuom) from mimiciii.inputevents_cv where itemid={0} group by hadm_id, amountuom\
 union all select hadm_id, amountuom, count(amountuom) from mimiciii.inputevents_mv where itemid={0} group by hadm_id, amountuom order by hadm_id'
for itemid in [x[0] for x in valid_vupairs_des[:14]]:
    cur = conn.cursor()
    cur.execute(sql.format(itemid))
    results = cur.fetchall()
    print('IN', itemid)
    print('hadm_id\t\tamountuom\tcount')
    for res in results:
        print('\t\t'.join(map(str, res)))
    print()

In [10]:
valid_vupairs = np.load('res/filtered_input_raw.npy').tolist()['raw']
valid_input = [x[0] for x in valid_vupairs]
valid_input_unit = [x[2][0][0] for x in valid_vupairs]
print(valid_input, valid_input_unit)

np.save('res/filtered_input.npy',{'id':valid_input,'unit':valid_input_unit})
print('saved!')

[225936, 30044, 46770, 42596, 44526, 43038, 40327, 225883, 43829, 43750, 225931, 43007, 40743, 30001, 43744, 41390, 30138, 45865, 46057, 44064, 46387, 222051, 45915, 44407, 43081, 45333, 42576, 46583, 45929, 46630, 45640, 42456, 44539, 45090, 43764, 46496, 46362, 45741, 44673, 45803, 46564, 45505, 225991, 42344, 30320, 41358, 45166, 41776, 45270, 43023, 44605, 41357, 30332, 44061, 43474, 40645, 44690, 30119, 44945, 42575, 46101, 41712, 45771, 221456, 46002, 225876, 44505, 46264, 30166, 42999, 45073, 45787, 46026, 44610, 45838, 45125, 44292, 42083, 225860, 46218, 42554, 227535, 41194, 43087, 46040, 45802, 30148, 42438, 42629, 44112, 45214, 40880, 45683, 44054, 44527, 45688, 44818, 45912, 45675, 45182, 45863, 45114, 45458, 42429, 30375, 41216, 45588, 45871, 30399, 45275, 41356, 41913, 45230, 44144, 45587, 46093, 46419, 225973, 40030, 42674, 45956, 221319, 44795, 43845, 45317, 44182, 41449, 42065, 43835, 45627, 43957, 42230, 45187, 46643, 44696, 46610, 42208, 45765, 41094, 40850, 45580, 4

In [8]:
print(len(admission_ids))

58976
